In [8]:
#!pip install cohere python-pptx crewai==0.28.8 crewai_tools==0.1.6 langchain_cohere unstructured_client unstructured[all-docs] chromadb langsmith -U

In [9]:
#pip install faiss-cpu

In [10]:
#!pip install langchainhub

In [37]:
#=================
# Import Libraries
#=================

from crewai import Agent, Task, Crew
from crewai_tools import BaseTool
import os


from langchain_cohere import ChatCohere
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.auto import partition
from unstructured_client import UnstructuredClient


from langchain_core.documents import Document
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from IPython.display import Markdown


In [12]:



#=================
# Input Files
#=================

file1 = 'unleashing-power-gen-ai-in-procurement.pdf'
file2 = 'UK_Tech_Clusters.pptx'
file3 = 'AI_In_Agriculture.docx'

#=================
# Unstructured Key
#=================

s = UnstructuredClient(
    api_key_auth="Unstructured_Key_Goes_Here"
)


In [ ]:

#=================
# Partitioning Input Files
#=================

#pdf file
elements1 = partition(filename=file1)

#pptx
elements2 = partition(filename=file2)

#docx
elements3 = partition(filename=file3)

In [ ]:

#=================
# Chunking Input Files By Title
#=================

elements = chunk_by_title(elements1 + elements2 + elements3 )

documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    metadata["source"] = metadata["filename"]
    documents.append(Document(page_content=element.text, metadata=metadata))

In [21]:
#=================
# Embedding Part
#=================

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

embeddings = hf

C:\Users\Mustafa\Anaconda_3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
#=================
# Vector Storage and Retrieval
#=================

vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()
handler =  StdOutCallbackHandler()

In [30]:
#=================
# Retrieval Chain
#=================

os.environ["openai_api_key"] = "Openai_Key_Goes_Here"

llm = ChatOpenAI(temperature=0.0,
                 model="gpt-4o",
                 max_tokens=512)

# Retrieval qa chain
qa = RetrievalQA.from_chain_type(
                                    llm=llm,
                                    chain_type="stuff",
                                    callbacks=[handler],
                                    retriever=retriever)

In [31]:
#=================
# Customized Agentic Tool
#=================

class MyCustomTool(BaseTool):
    name: str = "Research Paper"
    description: str = "Research Paper"

    def _run(self, argument: str) -> str:
        return qa.run("Please read, understand and analyze the provided documents")

research_tool = MyCustomTool()

In [32]:

#=================
# Crew Agents
#=================

researcher = Agent(
    role="Researcher",
    goal="Research and the documents",
    backstory="extract the main information and provide the main insights to the writer "
              "Don't provide any information outside the provided documents"
    ,
    verbose=True,
    tools=[research_tool],
    allow_delegation=False
    )

writer = Agent(
    role="Writer",
    goal= 'Craft easy to understand summary report',
    backstory="""You are working on writing a summary of the provided documents.
                  Use the content of the documents to develop
                  a short comprehensive summary""",
    verbose=True,
    allow_delegation=False,
)

#=================
# Crew Tasks
#=================

# Tasks
research = Task(
    description="""Conduct an in-depth analysis of the provided documents.
                    Identify the key components and try to understand how each works.""",
    expected_output="""our final answer must be a full analysis report of the documents""",
    agent=researcher

)



write = Task(
    description=(
        "1. Use the researcher's material to craft a compelling summary"
		    "2. Sections/Subtitles are properly named "
         "3. Limit the document to only 500 words "
    ),
    agent=writer,
    expected_output='a compelling summary report' 
)


#=================
# Execution
#=================

crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=2
)

result = crew.kickoff()



 [DEBUG]: == Working Agent: Researcher
 [INFO]: == Starting Task: Conduct an in-depth analysis of the provided documents.
                    Identify the key components and try to understand how each works.


> Entering new CrewAgentExecutor chain...
I need to begin by using the research tool to analyze the provided documents. 

Action: 
Research Paper

Action Input: 
{
  "argument": "Provided Documents"
}


C:\Users\Mustafa\Anaconda_3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.
 

The provided context emphasizes the importance of government involvement and collaboration among various stakeholders to ensure the responsible and sustainable integration of new technologies. Here are the key points:

1. **Government Involvement**: Governments play a crucial role in balancing promotion and control to ensure that new technologies are integrated responsibly and sustainably. This involves creating policies and regulations that encourage innovation while also mitigating potential risks.

2. **Collaboration and Stakeholder Engagement**: Effective integration of technologies requires collaboration between industry stakeholders, researchers, and policymakers. Inclusive decision-making processes are essential to consider the interests of all parties and to address complex challenges such as climate risk and food crises. This collaborative approach ensures that multiple perspectives and expertise are brought together 

In [33]:
from IPython.display import Markdown
Markdown(result)

The provided documents underscore the critical role of government participation, multi-stakeholder collaboration, and a comprehensive approach in the sustainable and responsible incorporation of emerging technologies.

Under the section **Government Involvement**, it is highlighted that governments are essential in striking a balance between encouraging and regulating new technologies. They are tasked with formulating policies and rules that stimulate innovation while also controlling potential dangers. This government intervention guarantees that technological integration takes place in a responsible and sustainable manner.

The second section, **Collaboration and Stakeholder Engagement**, stresses the necessity of efficient cooperation among industry participants, scientists, and policymakers for technology integration. Decision-making processes that are inclusive and consider all stakeholders' interests are vital for tackling intricate issues such as climate risk and food shortages. This cooperative methodology ensures that a variety of viewpoints and skills are employed to devise comprehensive plans.

The third section, **Holistic Approach**, advocates for a comprehensive perspective that considers both technological advancements and environmental sustainability. This approach involves assessing the broader consequences of technologies and ensuring that their growth and deployment are in line with sustainable practices.

In conclusion, as highlighted in the documents, integrating novel technologies necessitates a balanced, collaborative, and holistic approach. Active government participation and stakeholder engagement are crucial elements in addressing potential hazards and promoting sustainable growth.

In [34]:
'''
template = """Provide a precise and consice response to the below question from the provided context and don't use your own knowledge. 
If the context does not provide the necessary information to answer the question, then respond politely that you need more information.
Question: {question}
=========
Context: {context}
=========
"""
prompt = PromptTemplate(template=template, input_variables=[ "question","context"])
doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

question = 'Summarize each document in multiple separate paragraphs and mention the source document title and name at the beginning of each summary paragraph. limit each paragraph to 100 words'
response =qa_chain.invoke({
    "question": question,
    "chat_history": []
})["answer"]

#-------------------------------------------

from IPython.display import Markdown
Markdown(response)

#-------------------------------------------


template = """Act as an evaluator and evaluate of the LLM Response is relevant to the Question and free of hallucination. 
Also make sure that it's structured correctly where the source name is at the start of each paragraph.
Present your evaluation as a score from 1 to 10 where 10 is the highest in terms of relevance and structure.
Provide the reasons behind your statement.

Question: {question}
=========
Response: {response}
=========
Context: {context}
=========
"""
prompt = PromptTemplate(template=template, input_variables=["response", "context","question"])
doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)
response2 =qa_chain.invoke({
    "question": template,"response": response,"chat_history": []
})["answer"]

#------------------------------------------------

from IPython.display import Markdown
Markdown(response2)
'''

'\ntemplate = """Provide a precise and consice response to the below question from the provided context and don\'t use your own knowledge. \nIf the context does not provide the necessary information to answer the question, then respond politely that you need more information.\nQuestion: {question}\n=========\nContext: {context}\n=========\n"""\nprompt = PromptTemplate(template=template, input_variables=[ "question","context"])\ndoc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")\nquestion_generator_chain = LLMChain(llm=llm, prompt=prompt)\nqa_chain = ConversationalRetrievalChain(\n    retriever=retriever,\n    question_generator=question_generator_chain,\n    combine_docs_chain=doc_chain,\n)\n\nquestion = \'Summarize each document in multiple separate paragraphs and mention the source document title and name at the beginning of each summary paragraph. limit each paragraph to 100 words\'\nresponse =qa_chain.invoke({\n    "question": question,\n    "chat_history": []\n})